In [ ]:
import numpy as np
import matplotlib.pylab as plt

from AngleRealizability import angle_set

%reload_ext autoreload
%autoreload 2

# Create synthetic setups

In [ ]:
##### random -- too big
np.random.seed(7)
num_ambiguities = 2
num_drones = 5
gt_positions = np.random.rand(num_drones, 2)
gt_orientations = np.random.rand(num_drones) * 360

In [ ]:
##### random -- too big for brute force
np.random.seed(8)
num_ambiguities = 3
num_drones = 5
gt_positions = np.random.rand(num_drones, 2)
gt_orientations = np.random.rand(num_drones) * 360

In [ ]:
##### hard-coded
num_drones = 4
gt_positions = np.array([[0, 0], [1., 1.], [1., 0]]) # M x d
gt_orientations = [0, 0, 0]
if num_drones == 3:
    num_ambiguities = 2
elif num_drones == 4:
    num_ambiguities = 1
    gt_positions = np.r_[gt_positions, [[0, 1.]]]
    gt_orientations = np.r_[gt_orientations, 0]

In [ ]:
from angle_algorithms import get_direction

fig, ax = plt.subplots()
for i, (alpha, this_pos) in enumerate(zip(gt_orientations, gt_positions)):
    gt_other_idx = np.delete(np.arange(num_drones), i)
    ax.scatter(*this_pos, color=f'C{i}', label=f'p{i}')
    ax.quiver(*this_pos, *get_direction(alpha), color=f'C{i}') #x, y, u, w
    
    gt_others = gt_positions[gt_other_idx, :]
    for j, other_pos in zip(gt_other_idx, gt_others):
        other_dir = (other_pos - this_pos)
        other_dir /= np.linalg.norm(other_dir)
        ax.quiver(*this_pos, *other_dir, color=f'C{j}',
                  linestyle = ':') #x, y, u, w
ax.legend()
ax.axis('equal')
pass

In [ ]:
import itertools
np.random.seed(2)

def print_angles(abs_angles_dict):
    for corner, dict_ in abs_angles_dict.items():
        for other, val in dict_.items():
            val = np.array(val).reshape((-1, )) * 180 / np.pi
            val = val.astype(np.int)  #round(0)
            print(f"direction ({corner}, {other}): {val}")


abs_angles_dict = angle_set.get_absolute_angles(gt_positions, num_ambiguities=num_ambiguities)
print_angles(abs_angles_dict)

print('expected number per drone', (num_ambiguities + 1)**(num_drones - 1))
print('expected number of combinations:', ((num_ambiguities + 1)**(num_drones - 1))**(num_drones))

# Brute-force algorithm

In [ ]:
from angle_algorithms import * 
from log_progress import log_progress
import time# generate all possible choices for each drone:

def brute_force(abs_angles_dict):
    combis_per_drone = {}
    for drone, drone_abs_angle_dict in abs_angles_dict.items():
        list_of_lists = list(drone_abs_angle_dict.values())
        angle_combinations = list(itertools.product(*list_of_lists)) 
        combis_per_drone[drone] = angle_combinations
    print(f'number of combinations per drone: ', len(angle_combinations))

    indices_dict = {drone:range(len(combis)) 
                    for (drone, combis) in combis_per_drone.items()}
    list_of_lists = list(indices_dict.values())
    all_combinations = list(itertools.product(*list_of_lists))
    print('total number of combinations:', len(all_combinations))

    total = len(all_combinations)

    start = time.time()
    for combi in log_progress(all_combinations):
        #print(f'{i} / {total} ({i/total:.0f}%)')
        chosen_abs_angle_dict = {}
        for choice, (drone, combis_drone) in zip(combi, combis_per_drone.items()):
            chosen_abs_angle_dict[drone] = dict(zip(
                abs_angles_dict[drone].keys(),
                combis_drone[choice]))

        preprocess(chosen_abs_angle_dict)

        mae, __ = test_feasibility(chosen_abs_angle_dict)
        if mae < 1e-2:
            print(f'very small error error for choice {combi}: {mae:.1e}')
            print_angles(chosen_abs_angle_dict)

    print(f'done after {time.time() - start:.2f} seconds')

brute_force(abs_angles_dict)

# Greedy algorithm

In [ ]:
# find the correct answer using brute force.
from backtrack import constraints_check, PartialSolution

import copy
P = copy.deepcopy(abs_angles_dict)
print_angles(P)

if num_drones == 3:
    c = PartialSolution()
    indices = [2, 1, 2, 2, 2, 2]
    c.create_solution(P, indices)
    mae, message = constraints_check(P, c)
    print(message)
    #brute_force(P)
elif num_drones > 3:
    c = PartialSolution()
    indices = 3 * np.ones((20,), dtype=int)
    c.create_solution(P, indices, num_drones=num_drones)
    mae, message = constraints_check(P, c)
    print(message)
    #brute_force(P)

In [ ]:
from backtrack import backtrack, root
import time
start = time.time()
backtrack(P, root(P))
print(f'done after {time.time() - start:.2f} seconds')